# Scraping From Boomplay

In [1]:
#import libraries

from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
from datetime import datetime
import re
from matplotlib import pyplot as plt
import seaborn as sb

In [2]:
#functions
def append(value, destination):
    '''This function appends values to a list '''
    value.append(destination)

def DataFrame(col1,col2,col3,col4):
    '''This function collects dictionaries and turns the
     dictionaries into a dataframes and merges the datafrmaes to form
     a single dataframe'''


    #first_c = pd.DataFrame(col1)
    #second_c = pd.DataFrame(col2)
    #third_c = pd.DataFrame(col3)
    #forth_c = pd.DataFrame(col4)

    df = col1|col2|col3|col4 #merging the dataframe

    return df

In [3]:
#### SCRAPE BOOMPLAY FOR SONGS AND ARTISTS

url='https://www.boomplay.com/playlists/26356675?from=home' #The URL


response = requests.get(url)
#response
soup= BeautifulSoup(response.content, 'lxml')
soup

<!DOCTYPE html>
<html lang="en">
<head>
<link href="https://source.boomplaymusic.com/" rel="preconnect"/>
<link href="https://www.gstatic.com/" rel="preconnect"/>
<link href="https://source.boomplaymusic.com/" rel="dns-prefetch"/>
<link href="https://www.google-analytics.com" rel="dns-prefetch"/>
<link href="https://stats.g.doubleclick.net/" rel="dns-prefetch"/>
<link href="https://www.google.com" rel="dns-prefetch"/>
<link href="https://tagmanager.google.com/" rel="dns-prefetch"/>
<link href="https://music-202.boomplaymusic.com/" rel="dns-prefetch"/>
<link href="https://recaptcha.net/" rel="preconnect"/>
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-TYGC13V7V3"></script>
<script>window.dataLayer = window.dataLayer || [];

    function gtag() {
        dataLayer.push(arguments);
    }

    gtag('js', new Date());
    gtag('config', 'G-TYGC13V7V3');
    gtag('config', 'UA-184500473-2');</script>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X

In [4]:
#Dictionaries
song_name = {'song_name':[]}
artist = {'artist_name':[]}
time= []


#Getting the Song title and artist name 
#Song name
artist_list = []

song_data = soup.findAll('a', {'class':'songName'})
artist_data= soup.findAll('a', {'class':'artistName'})


index=0
while index < 104:
    name = song_data[index].text.replace('\n','').split('ft.')
    if len(name)>=2: 
        name=name[0]
        #print(name)
    else:
        name = name[0]
        #print(name)

    append(song_name['song_name'],name) #appending to song_name dict
    index = index+1

In [5]:
#song title    
i = 0
while i<104:
    artist_name = artist_data[i].text
    artist['artist_name'].append(artist_name)
    artist_list.append(artist_name)
    i = i+1

In [6]:
# #### Programatic web search for artist's age

#loop for building the url
age_url =[]

for i in artist_list:
    #first_C ="'"
    #second_C ="'"
    url2= "https://en.wikipedia.org/wiki/"+ i
    
    append(age_url,url2)    #append to age_url

In [7]:
#getting and storing the artist age
error = []
artist_age = {'a_age':[]}
n=0
pattern = "[0-9]{4}|[0-9]{2}|[0-9]{2}"
for j in age_url:
    
    response1 = requests.get(j)

    try:
        soup1 = BeautifulSoup(response1.content,'html.parser')
        age_data = soup1.select_one('span span',{'class':'bday'})
        age = age_data.text
       #checking if the the age data mathes the pattern 
        if (re.search(pattern, age)):
            artist_age['a_age'].append(age)
        else:
            artist_age['a_age'].append('NA')

    except AttributeError:
        artist_age['a_age'].append('NA')
        error.append('NA')              



pd.DataFrame(artist_age)

,a_age
0,1996-01-29
1,NA
2,. Retrieved 2016-12-10
3,1995-02-16
4,1995-01-13
...,...
99,NA
100,1997-05-19
101,1990-07-12
102,1992-10-19


In [8]:
#Getting run_time
time = {'time':[]}

for t in range(104):
    time_data = soup.findAll('li',{'class':'clearfix play_one'})
    append(time['time'],time_data[t].time.text)



### Creating DataFrames



df = DataFrame(song_name,artist, artist_age, time)
df = pd.DataFrame(df)

In [11]:
# #### Cleaning
#changing a_age column to datetime datatime

df.a_age = pd.to_datetime(df.a_age, errors='coerce', format='%Y-%m-%d')
df.a_age

0     1996-01-29
1            NaT
2            NaT
3     1995-02-16
4     1995-01-13
         ...    
99           NaT
100   1997-05-19
101   1990-07-12
102   1992-10-19
103   1991-09-05
Name: a_age, Length: 104, dtype: datetime64[ns]

In [87]:
df.to_csv("Boomplay Scraped songs.csv")

In [13]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [14]:
df

,song_name,artist_name,a_age,time
0,Like Ice Spice,Blaqbonez,1996-01-29,02:03
1,Defender,Frankie Jay,NaT,02:04
2,Ngozi,Crayon,NaT,03:41
3,Sharpally,Young Jonn,1995-02-16,02:43
4,Lonely At The Top,Asake,1995-01-13,02:37
5,Honey,Zuchu,NaT,02:19
6,Babylon,Patoranking,1990-05-27,02:57
7,Mahaba,Alikiba,NaT,03:16
8,Sele,Mbosso,1991-10-03,04:12
9,Alone (Speed Up),Neeja,NaT,01:20
